In [9]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy.engine import url
import json
from sqlalchemy import extract
from sqlalchemy.engine import make_url
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import psycopg2

In [10]:
## I included really anything in here if you actually wanted to run sql queries.

In [13]:
# opening configuration file, saving into a variable, then establishing that variable as env which specifies the data as development
# environment credentials
with open('sql/config.json') as datafile:
    data = json.load(datafile)

env = data['dev']
print(env)

{'db': 'CRE_COVID_DB', 'user': 'postgres', 'pass': 'Markishot23!', 'host': '127.0.0.1', 'port': '5432'}


In [12]:
# Instantiating the environment column values as the column names to hide sensitive data
db = env['db']
user = env['user']
password = env['pass']
port = env['port']
host = env['host']

In [5]:
# Connection string
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [6]:
# Reflecting an existing DB into a new model
Base = automap_base()

In [7]:
Base.prepare(autoload_with = engine)

In [8]:
# Seeing what tables there are
Base.classes.keys()

['races', 'cases', 'jhu_cre_cases_deaths', 'location']

In [9]:
# Saving the tables as references here if u want 
location = Base.classes.location

In [10]:
# Another table reference 
jhu_data = Base.classes.jhu_cre_cases_deaths

In [11]:
# Session object is the handler to the database, estb convo w db.
# Sessionmaker class creates a 'top level' session configuration that then can be used throughout the application without
# the need to repeat config arguments.'- Credit to rfkortekaas on Stack for explaining.
Session = sessionmaker(bind = engine)
session = Session()

In [12]:
# Test query to make sure database connection works
sql = session.query(jhu_data)

In [13]:
# An example of using sql commands:
sql = '''
    SELECT * FROM location;
'''

with engine.connect() as conn:
    query = conn.execute(text(sql))
df = pd.DataFrame(query.fetchall())

In [14]:
df

,fips,lat,long,state,county
0,1001,32.539527,-86.644082,Alabama,Autauga County
1,1003,30.727750,-87.722071,Alabama,Baldwin County
2,1005,31.868263,-85.387129,Alabama,Barbour County
3,1007,32.996421,-87.125115,Alabama,Bibb County
4,1009,33.982109,-86.567906,Alabama,Blount County
...,...,...,...,...,...
3137,56037,41.659439,-108.882788,Wyoming,Sweetwater County
3138,56039,43.935225,-110.589080,Wyoming,Teton County
3139,56041,41.287818,-110.547578,Wyoming,Uinta County
3140,56043,43.904516,-107.680187,Wyoming,Washakie County


In [15]:
# An example of using sql commands:
sql = '''
    SELECT * FROM jhu_cre_cases_deaths;
'''

with engine.connect() as conn:
    query = conn.execute(text(sql))
df2 = pd.DataFrame(query.fetchall())

In [16]:
df2

,fips,cases,deaths,lat,long,state,county,popuni,total_population,zero_rf,...,single_fathers_pop,plus_family_homes,highschool_grad,multilingual_5yrs_plus,full_time_workers,no_health_insur,internet_homes,no_vehicle,homeowner_vacancy,rental_vacancy
0,1001,19732,230,32.539527,-86.644082,Alabama,Autauga County,55688,55380,20573,...,723,111,49283,779,29291,3953,44884,2450,779,1726
1,1003,69641,724,30.727750,-87.722071,Alabama,Baldwin County,221898,212830,78622,...,2218,887,201483,3994,112058,19748,181512,7322,7322,96747
2,1005,7451,103,31.868263,-85.387129,Alabama,Barbour County,22023,25361,5024,...,220,132,16120,572,8170,2488,13323,2246,836,1629
3,1007,8067,109,32.996421,-87.125115,Alabama,Bibb County,20393,22493,6280,...,346,163,16130,265,8096,2182,14111,1264,305,1142
4,1009,18616,261,33.982109,-86.567906,Alabama,Blount County,57697,57681,18189,...,1038,115,46446,1961,27463,6231,42118,2365,403,4846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,12484,139,41.659439,-108.882788,Wyoming,Sweetwater County,41888,43521,16977,...,502,209,38872,1298,21739,4733,35185,712,879,12817
3138,56039,12123,16,43.935225,-110.589080,Wyoming,Teton County,23390,23280,7250,...,140,771,22267,1777,13028,2970,20372,561,467,5730
3139,56041,6378,43,41.287818,-110.547578,Wyoming,Uinta County,20183,20479,7744,...,322,141,18709,322,10091,2260,18063,686,100,2401
3140,56043,2749,50,43.904516,-107.680187,Wyoming,Washakie County,7738,8027,2601,...,108,77,6948,108,3869,1160,6051,379,15,1903


In [17]:
df_copy = df2
df_copy.columns

Index(['fips', 'cases', 'deaths', 'lat', 'long', 'state', 'county', 'popuni',
       'total_population', 'zero_rf', 'one_two_rf', 'three_rf',
       'housing_units', 'hispanic_pop', 'white_pop', 'black_pop', 'native_pop',
       'asian_pop', 'pacific_islander_pop', 'other_race_pop',
       'bi_tri_racial_pop', 'male_pop', 'female_pop', 'veteran',
       'gini_ind_income', 'rural_pop', 'median_age_pop', 'elder_pop',
       'disability_pop', 'below_poverty_level', 'single_mothers_pop',
       'single_fathers_pop', 'plus_family_homes', 'highschool_grad',
       'multilingual_5yrs_plus', 'full_time_workers', 'no_health_insur',
       'internet_homes', 'no_vehicle', 'homeowner_vacancy', 'rental_vacancy'],
      dtype='object')

In [18]:
sql = '''
        DROP TABLE IF EXISTS races CASCADE;

        CREATE TABLE races (
            fips int,
            hispanic_pop int,
            white_pop int,
            black_pop int,
            native_pop int,
            asian_pop int, 
            pacific_islander_pop int,
            other_race_pop int,
            bi_tri_racial_pop int,
            PRIMARY KEY (fips)
        );
        
        INSERT INTO races (fips, hispanic_pop, white_pop, black_pop, native_pop, asian_pop, pacific_islander_pop, other_race_pop, bi_tri_racial_pop)
        SELECT jhu_cre_cases_deaths.fips, jhu_cre_cases_deaths.hispanic_pop, 
            jhu_cre_cases_deaths.white_pop, jhu_cre_cases_deaths.black_pop, jhu_cre_cases_deaths.native_pop,
            jhu_cre_cases_deaths.asian_pop, jhu_cre_cases_deaths.pacific_islander_pop, jhu_cre_cases_deaths.other_race_pop,
            jhu_cre_cases_deaths.bi_tri_racial_pop
        FROM jhu_cre_cases_deaths;
        
        SELECT * FROM races

'''

with engine.connect() as conn:
    query = conn.execute(text(sql))
output = pd.DataFrame(query.fetchall())
output

,fips,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_islander_pop,other_race_pop,bi_tri_racial_pop
0,1001,1559,41543,10580,167,556,0,111,1169
1,1003,10207,184397,20414,1553,1997,0,443,3328
2,1005,969,10086,10438,66,110,0,88,264
3,1007,530,15192,4506,20,20,0,0,101
4,1009,5365,50138,865,57,230,0,230,865
...,...,...,...,...,...,...,...,...,...
3137,56037,6660,33342,460,418,293,0,0,670
3138,56039,3508,19016,280,70,304,23,0,163
3139,56041,1836,17660,20,141,40,0,0,484
3140,56043,1098,6337,0,38,0,0,15,239


In [19]:
sql = '''
        DROP TABLE IF EXISTS cases;

        CREATE TABLE cases (
            fips int,
            cases int,
            deaths int,
            PRIMARY KEY (fips),
            FOREIGN KEY (fips) REFERENCES races(fips)
        );
        
        INSERT INTO cases (fips, cases, deaths)
        SELECT jhu_cre_cases_deaths.fips, jhu_cre_cases_deaths.cases, jhu_cre_cases_deaths.deaths
        FROM jhu_cre_cases_deaths;
        
        SELECT * FROM cases

'''

with engine.connect() as conn:
    query = conn.execute(text(sql))
output = pd.DataFrame(query.fetchall())
output

,fips,cases,deaths
0,1001,19732,230
1,1003,69641,724
2,1005,7451,103
3,1007,8067,109
4,1009,18616,261
...,...,...,...
3137,56037,12484,139
3138,56039,12123,16
3139,56041,6378,43
3140,56043,2749,50


In [20]:
sql = '''
        SELECT cases.fips, cases.cases, cases.deaths, races.white_pop, races.black_pop, races.native_pop, races.asian_pop, races.pacific_islander_pop, races.bi_tri_racial_pop, races.other_race_pop
        FROM cases
        INNER JOIN races ON cases.fips = races.fips;
'''

with engine.connect() as conn:
    query = conn.execute(text(sql))
ouput = pd.DataFrame(query.fetchall())
ouput

,fips,cases,deaths,white_pop,black_pop,native_pop,asian_pop,pacific_islander_pop,bi_tri_racial_pop,other_race_pop
0,1001,19732,230,41543,10580,167,556,0,1169,111
1,1003,69641,724,184397,20414,1553,1997,0,3328,443
2,1005,7451,103,10086,10438,66,110,0,264,88
3,1007,8067,109,15192,4506,20,20,0,101,0
4,1009,18616,261,50138,865,57,230,0,865,230
...,...,...,...,...,...,...,...,...,...,...
3137,56037,12484,139,33342,460,418,293,0,670,0
3138,56039,12123,16,19016,280,70,304,23,163,0
3139,56041,6378,43,17660,20,141,40,0,484,0
3140,56043,2749,50,6337,0,38,0,0,239,15


In [21]:
sql = '''
        DROP TABLE IF EXISTS case_race;

        SELECT cases.fips, cases.cases, cases.deaths, races.white_pop, races.black_pop, races.native_pop, races.asian_pop, races.pacific_islander_pop, races.bi_tri_racial_pop, races.other_race_pop
        INTO case_race
        FROM cases
        INNER JOIN races ON cases.fips = races.fips;
        
        SELECT * FROM cases
'''

with engine.connect() as conn:
    query = conn.execute(text(sql))
ouput = pd.DataFrame(query.fetchall())
ouput

,fips,cases,deaths
0,1001,19732,230
1,1003,69641,724
2,1005,7451,103
3,1007,8067,109
4,1009,18616,261
...,...,...,...
3137,56037,12484,139
3138,56039,12123,16
3139,56041,6378,43
3140,56043,2749,50
